In [1]:
import torch
import os
from tqdm import tqdm

from src.utils import save_images
from src.model.unet import UNet

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = UNet().to(device)
model.load_state_dict(torch.load("checkpoints/DDPM_Uncondtional_cat/ckpt_395.pt", weights_only=True))

<All keys matched successfully>

In [6]:
def sample(model, img_size, noise_steps, alpha, alpha_hat, beta, name_dir):
    model.eval()
    os.makedirs(os.path.join("results", name_dir), exist_ok=True)

    with torch.no_grad():
        x = torch.randn((1, 3, img_size, img_size)).to(device)
        for i in tqdm(reversed(range(1, noise_steps)), position=0):
            t = (torch.ones(1) * i).long().to(device)
            # Debug print to ensure t is within valid range
            print(f"t: {t.item()}, max index: {alpha.size(0)-1}")

            if t.item() >= alpha.size(0):
                print("Index out of range. Breaking.")
                break
            
            predicted_noise = model(x, t)
            alpha_val = alpha[t][:, None, None, None]
            alpha_hat_val = alpha_hat[t][:, None, None, None]
            beta_val = beta[t][:, None, None, None]

            if i > 1:
                noise = torch.randn_like(x)
            else:
                noise = torch.zeros_like(x)

            x = 1 / torch.sqrt(alpha_val) * (x - ((1 - alpha_val) / (torch.sqrt(1 - alpha_hat_val))) * predicted_noise) + torch.sqrt(beta_val) * noise
    return x
            
            #img_to_save = x.clamp(-1, 1) # Keep this tensor in float32 for the model processing
            #save_image = (img_to_save * 127.5 + 127.5).type(torch.uint8)  # Convert to byte only for saving
            #save_images(save_image, os.path.join("results", name_dir, f"sample_{i}.png"))

In [7]:
beta = torch.linspace(1e-4, 0.02, 1000).to(device).float()
alpha = 1 - beta
alpha_hat = torch.cumprod(alpha, dim=0).float()


sample(model=model,
       img_size=64,
       noise_steps=1000,
       alpha=alpha,
       alpha_hat=alpha_hat,
       beta=beta,
       name_dir="Cat_Step")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
